# Wegpunkte
## TODO
Beschreibung wie man die Wegpunkte macht.
Aber eigentlich mach ich nur ne Line mit JOSM, geb der "find_me" = "irgendein identifier", exportier das osm, suche den identifier und gebe die wegpunkte in yaml aus.

In [ ]:
import osmnx as ox
import pyproj # we need this for jupyter-book to build right
import yaml
from geopy.distance import geodesic

In [ ]:
def calculate_track_length(coordinates):
    track_length = 0.0
    for i in range(len(coordinates) - 1):
        start_point = (coordinates[i]['latitude'], coordinates[i]['longitude'])
        end_point = (coordinates[i+1]['latitude'],
                     coordinates[i+1]['longitude'])
        track_length += geodesic(start_point, end_point).meters
    return track_length

In [ ]:
geo_df = ox.features_from_xml(
    "out/Testing GPS Waypoint making-Schkeuditz.osm")


geo_df.head()

In [ ]:
filtered_nodes = geo_df[geo_df['findme'].notna()]
filtered_nodes = filtered_nodes.dropna(axis=1, how='all')
filtered_nodes

In [ ]:
filtered_nodes.iloc[0].geometry
coordinates = list(filtered_nodes.iloc[0].geometry.coords)
data = [{'latitude': lat, 'longitude': lon, 'yaw': 0.0}
        for lon, lat in coordinates]
data

In [ ]:

# Get the unique ways from the filtered_nodes GeoDataFrame
unique_ways = filtered_nodes.index.get_level_values('osmid').unique()

# Iterate over each unique way
for way in unique_ways:
    linestring = filtered_nodes.loc[('way', way), 'geometry']
    # print(linestring)
    coordinates = list(linestring.coords)
    data = [{'latitude': lat, 'longitude': lon, 'yaw': 0.0}
            for lon, lat in coordinates]
    length = calculate_track_length(data)
    # Create a YAML file path for the current way
    name = filtered_nodes.loc[('way', way), 'findme']
    yaml_file_path = f'out/way_{name}.yaml'
    print(f'Writing coordinates for way {name} to {yaml_file_path}')
    print(f'Length of the track: {length} meters')
    # Write the coordinates to the YAML file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False)

In [ ]:
reverse = ["Rathaus zu Bürgeramt"]
for way in reverse:
    way_df = filtered_nodes[filtered_nodes['findme'] == way]
    linestring = way_df.iloc[0].geometry
    # linestring = filtered_nodes.loc[('findme', way), 'geometry']
    # print(linestring)
    coordinates = list(linestring.coords)
    data = [{'latitude': lat, 'longitude': lon, 'yaw': 0.0}
            for lon, lat in coordinates]
    length = calculate_track_length(data)
    # Create a YAML file path for the current way
    # name = filtered_nodes.loc[('way', way), 'findme']
    name = way
    reversed_name = name.split(' zu ')[::-1]
    reversed_name = ' zu '.join(reversed_name)
    print(reversed_name)
    name = reversed_name
    yaml_file_path = f'out/way_{name}.yaml'
    data.reverse()
    print(f'Writing coordinates for way {name} to {yaml_file_path}')
    print(f'Length of the track: {length} meters')
    # Write the coordinates to the YAML file
    with open(yaml_file_path, 'w') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False)

# Abschnitte
- Rathaus -> Bürgeramt
- Bürgeramt -> Rathaus
- Bürgeramt -> Rathausplatz
- Rathausplatz -> Ringstraße
- Ringstraße -> Amtsgasse
- Ringstraße -> Markt -> Amtsgasse
- Rathaus -> Post

## Volle Runde
- Route
    - Rathaus -> Bürgeramt -> Rathausplatz -> Ringstraße -> Amtsgasse -> Markt -> Rathaus -> Post -> Rathaus
    - Rathaus zu Bürgeramt
    - Bürgeramt zu Rathausplatz
    - Rathausplatz zu Ringstraße
    - Ringstraße zu Amtsgasse (über markt)
    - Amtsgasse zu Ringstraße (über markt) (reverse)
    - Ringstraße zu Rathausplatz (reverse)
    - Rathausplatz zu Rathaus (reverse)
    - Rathaus zu Post
- Alternative:
    - Rathaus -> Bürgeramt -> Rathausplatz -> Ringstraße -> Amtsgasse -> Rathaus -> Post -> Rathaus
    - Rathaus zu Bürgeramt
    - Bürgeramt zu Rathausplatz
    - Rathausplatz zu Ringstraße
    - Ringstraße zu Amtsgasse
    - Amtsgasse zu Ringstraße (reverse)
    - Ringstraße zu Rathausplatz (reverse)
    - Rathausplatz zu Rathaus (reverse)
    - Rathaus zu Post


In [ ]:
# o = filtered_nodes[filtered_nodes['findme'] == "Rathausplatz zu Ringstraße"]
# o.iloc[0]

In [ ]:
def get_coords(df, way_name, reverse=False):
    way_df = df[df['findme'] == way_name]
    coordinates = list(way_df.iloc[0].geometry.coords)
    data = [{'latitude': lat, 'longitude': lon, 'yaw': 0.0}
            for lon, lat in coordinates]
    if reverse:
        data.reverse()
    return data

In [ ]:
a = ["Rathaus zu Bürgeramt", "Bürgeramt zu Rathausplatz",
     "Rathausplatz zu Ringstraße", "Ringstraße zu Amtsgasse über Markt"]
b = ["Ringstraße zu Amtsgasse über Markt", "Rathausplatz zu Ringstraße",
     "Rathaus zu Rathausplatz"]  # all reversed
c = ["Rathaus zu Post"]
d = ["Rathaus zu Post"]  # reversed
full_route = []
for entry in a:
    full_route += get_coords(filtered_nodes, entry)
for entry in b:
    full_route += get_coords(filtered_nodes, entry, reverse=True)
for entry in c:
    full_route += get_coords(filtered_nodes, entry)
for entry in d:
    full_route += get_coords(filtered_nodes, entry, reverse=True)
full_route

In [ ]:
length = calculate_track_length(full_route)
print(f'Writing coordinates for way {name} to {yaml_file_path}')
print(f'Length of the track: {length} meters')

In [ ]:
with open("out/way_full_route.yaml", 'w') as yaml_file:
    yaml.dump(full_route, yaml_file, default_flow_style=False)

In [ ]:
a = ["Rathaus zu Bürgeramt", "Bürgeramt zu Rathausplatz",
     "Rathausplatz zu Ringstraße", "Ringstraße zu Amtsgasse"]
b = ["Ringstraße zu Amtsgasse", "Rathausplatz zu Ringstraße",
     "Rathaus zu Rathausplatz"]  # all reversed
c = ["Rathaus zu Post"]
d = ["Rathaus zu Post"]  # reversed
full_route_alternative = []
for entry in a:
    full_route_alternative += get_coords(filtered_nodes, entry)
for entry in b:
    full_route_alternative += get_coords(filtered_nodes, entry, reverse=True)
for entry in c:
    full_route_alternative += get_coords(filtered_nodes, entry)
for entry in d:
    full_route_alternative += get_coords(filtered_nodes, entry, reverse=True)
full_route_alternative

In [ ]:
length = calculate_track_length(full_route_alternative)
print(f'Writing coordinates for way {name} to {yaml_file_path}')
print(f'Length of the track: {length} meters')

In [ ]:
with open("out/way_full_route_alternative.yaml", 'w') as yaml_file:
    yaml.dump(full_route_alternative, yaml_file, default_flow_style=False)